# Mavlink  Client and Server
> Mavlink classes for Client and server.
> The server is on the companion computer and the client is on the ground station PC.

[https://mavlink.io/en/mavgen_python/](https://mavlink.io/en/mavgen_python/)
[https://www.ardusub.com/developers/pymavlink.html](https://www.ardusub.com/developers/pymavlink.html)

https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_TYPE



In [ ]:
#| default_exp mavlink.cam_client_server

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
# import logging
# logging.getLogger("uav").setLevel(logging.DEBUG)
# logging.root.setLevel(logging.INFO)
# import threading
# 
# import time, os
# # Set the environment variable before from pymavlink import mavutil  library is imported
# os.environ['MAVLINK20'] = '1'
from pymavlink import mavutil
from UAV.mavlink.base import MavLinkBase , UAV_SYSTEM_VEHICLE_ID, UAV_SYSTEM_GCS_CLIENT_ID
# from UAV.imports import *   # TODO why is this relative import on nbdev_export?


In [ ]:
# logging.getLogger("uav").setLevel(logging.INFO)

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
class CamClient(MavLinkBase):
 
    def __init__(self, connection_string, # "udpin:localhost:14550"
                 baudrate=57600, #baud rate of the serial port
                 mav_type=mavutil.mavlink.MAV_TYPE_GCS, # type used in heartbeat
                 server_system_ID=UAV_SYSTEM_VEHICLE_ID,  # remote or air uav system   1 = vehicle
                 client_system_ID=UAV_SYSTEM_GCS_CLIENT_ID,  # GCS system   255 = GCS
                 debug=False, # logging level
                 ):    
        super(CamClient, self).__init__( connection_string = connection_string,
                baudrate=baudrate, #baud rate of the serial port
                server_system_ID= server_system_ID,  # remote or air uav system   1 = vehicle
                client_system_ID= client_system_ID,  # GCS system   255 = GCS
                debug=debug, # logging level                   
                )
        self.client(mav_type=mav_type) # run as client

    def trigger_camera(self, camera_id:int=1): # camera id (0 for all cams)
        """
        Use MAV_CMD_DO_DIGICAM_CONTROL to trigger a camera 
        """
        self.send_command(mavutil.mavlink.MAV_CMD_DO_DIGICAM_CONTROL, 
                          [camera_id,  # param1 (session)  or cam # (0 for all cams)
                           1,  # param2 (trigger capture)
                           0,  # param3 (zoom pos)
                           0,  # param4 (zoom step)
                           0,  # param5 (focus lock)
                           0,  # param6 (shot ID)
                           0,  # param7 (command ID)
                          ])
        self.wait_ack()
        self.log.info("Camera triggered")
        
        
class CamServer(MavLinkBase):
    def __init__(self, connection_string, # "udpin:localhost:14550"
             baudrate=57600, #baud rate of the serial port
             mav_type=mavutil.mavlink.MAV_TYPE_CAMERA, # type used in heartbeat
             server_system_ID=UAV_SYSTEM_VEHICLE_ID,  # remote or air uav system   1 = vehicle
             client_system_ID=UAV_SYSTEM_GCS_CLIENT_ID,  # GCS system   255 = GCS
             debug=False, # logging level
             ):    
        super(CamServer, self).__init__( connection_string = connection_string,
                baudrate=baudrate, #baud rate of the serial port
                server_system_ID= server_system_ID,  # remote or air uav system   1 = vehicle
                client_system_ID= client_system_ID,  # GCS system   255 = GCS
                debug=debug, # logging level
                )
        self.num_commands_received = 0
        self.num_acks_received = 0
        self.server(mav_type=mav_type) # run as server

def test_cam_client_server():
    from fastcore.test import test_eq
    with CamClient("udpin:localhost:14445", debug=False) as client:
        with CamServer("udpout:localhost:14445", debug=False) as server:
            client.wait_heartbeat()

            for i in range(5):
                client.trigger_camera(2)
                server._test_command(2)

    print()
    print(f"server sys: {server.source_system};  msgs: {server.message_cnts}")
    print(f"client sys: {client.source_system};  msgs: {client.message_cnts}")

    test_eq(server.server_system_ID, server.source_system)
    test_eq(client.client_system_ID, client.source_system)
    test_eq(client.num_commands_sent, server.num_commands_received)
    test_eq(client.num_acks_received, server.num_commands_received)
    assert 'HEARTBEAT' in client.message_cnts
    
#test_cam_client_server()

In [ ]:
show_doc(CamClient)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/cam_client_server.py#L21){target="_blank" style="float:right; font-size:smaller"}

### CamClient

>      CamClient (connection_string, baudrate=57600, mav_type=6, debug=False)

Mavlink Camera Base

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| connection_string |  |  | "udpin:localhost:14550" |
| baudrate | int | 57600 | baud rate of the serial port |
| mav_type | int | 6 | type used in heartbeat |
| debug | bool | False | logging level |

In [ ]:
show_doc(CamClient.trigger_camera)


---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/cam_client_server.py#L34){target="_blank" style="float:right; font-size:smaller"}

### CamClient.trigger_camera

>      CamClient.trigger_camera (camera_id:int=1)

Use MAV_CMD_DO_DIGICAM_CONTROL to trigger a camera

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| camera_id | int | 1 | camera id (0 for all cams) |

In [ ]:
show_doc(CamServer)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/cam_client_server.py#L51){target="_blank" style="float:right; font-size:smaller"}

### CamServer

>      CamServer (connection_string, baudrate=57600, mav_type=30, debug=False)

Mavlink Camera Base

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| connection_string |  |  | "udpin:localhost:14550" |
| baudrate | int | 57600 | baud rate of the serial port |
| mav_type | int | 30 | type used in heartbeat |
| debug | bool | False | logging level |

In [ ]:
# show_doc(CamServer....)

#### Test locally using the same UDP ports
**CamClient** is set to `udpin:localhost:14445` and `CamServer` is set to the same `udpout:localhost:14445`
`udpin` is so that the client can receive UDP in at `localhost:1445`. The client will use another port to send UDP outwards.

In [ ]:
# from UAV.mavlink.cam_client_server  import CamClient, CamServer
from fastcore.test import *

with CamClient("udpin:localhost:14445", debug=False) as client:
    with CamServer("udpout:localhost:14445", debug=False) as server:
        client.wait_heartbeat()

        for i in range(5):
            client.trigger_camera(2)
            server._test_command(2)


print(f"client.num_commands_sent: {client.num_commands_sent}")
print(f"server.num_commands_received: {server.num_commands_received}")
print(f"client.num_acks_received: {client.num_acks_received}")

test_eq(client.num_commands_sent, server.num_commands_received)
test_eq(client.num_acks_received, server.num_commands_received)

INFO   | uav.CamClient        | 19:47:47.536 |[   base.py:106] MainThread | Starting MAVLink connection... Mavlink version 2 = True
INFO   | uav.CamClient        | 19:47:47.537 |[   base.py:118] MainThread | Source system Set: 200, Source component: 25
INFO   | uav.CamClient        | 19:47:47.537 |[   base.py:119] MainThread | Target system Set: 1, Target component: 100
INFO   | uav.CamClient        | 19:47:47.638 |[   base.py:270] Thread-26 (listen) | Listening for MAVLink commands from system: 1...
INFO   | uav.CamServer        | 19:47:47.639 |[   base.py:106] MainThread | Starting MAVLink connection... Mavlink version 2 = True
INFO   | uav.CamServer        | 19:47:47.640 |[   base.py:118] MainThread | Source system Set: 1, Source component: 100
INFO   | uav.CamServer        | 19:47:47.640 |[   base.py:119] MainThread | Target system Set: 200, Target component: 25
INFO   | uav.CamServer        | 19:47:47.741 |[   base.py:142] Thread-27 (send_heartbeat) | Starting heartbeat
INFO   | u

client.num_commands_sent: 5
server.num_commands_received: 5
client.num_acks_received: 5


This will show on wireshark as follows:
> Using the display filte string `not icmp  && udp.port eq 14445 && mavlink_proto`

![](images/wireshark_udp:14445.png)

For debugging help see [http://localhost:3000/tutorials/mavlink_doc&debug.html](http://localhost:3000/tutorials/mavlink_doc&debug.html)
 and [http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging](http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging)


In [ ]:
 # Todo add a link to the docs

#### Test using a Pixhawk connected via telemetry 2 and USB serial ports.
**CamClient** is set to `udpin:localhost:14445` and `CamServer` is set to `udpout:localhost:14435`
`udpin` is so that the client can receive UDP from the mavproxy server at `localhost:14445`
>`mavproxy.py  --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445`
>`mavproxy.py  --master=/dev/ttyACM3 --baudrate 57600 --out udpout:localhost:14435`

In [ ]:
# from UAV.mavlink.cam_client_server import  CamClient, CamServer
# from fastcore.test import *
# import time

debug = False
with CamServer("/dev/ttyUSB0", debug=debug ) as server:
    with CamClient("/dev/ttyACM1", debug=debug ) as client:
        print(client._log.level)
        client.wait_heartbeat(timeout=1)
        for i in range(3):
            client.trigger_camera(2)
            # server._test_command(1)
            # time.sleep(.15)

print(f"server.num_commands_received: {server.num_commands_received}")
print(f"client.num_commands_sent: {client.num_commands_sent}")
print(f"client.num_acks_received: {client.num_acks_received}")
print(f"server msgs: {server.message_cnts}")
print(f"client msgs: {client.message_cnts}")
test_eq (server.num_commands_received, client.num_commands_sent)
test_eq (server.num_commands_received, client.num_acks_received)
test_eq (server.message_cnts['COMMAND_LONG'], client.message_cnts['COMMAND_ACK'])
assert 'HEARTBEAT' in client.message_cnts

INFO   | uav.CamServer        | 20:20:03.952 |[   base.py:105] MainThread | Starting MAVLink connection... Mavlink version 2 = True
INFO   | uav.CamServer        | 20:20:03.953 |[   base.py:117] MainThread | Source system Set: 1, Source component: 100
INFO   | uav.CamServer        | 20:20:03.954 |[   base.py:118] MainThread | Target system Set: 200, Target component: 25
INFO   | uav.CamServer        | 20:20:04.055 |[   base.py:155] Thread-32 (send_heartbeat) | Starting heartbeat
INFO   | uav.CamServer        | 20:20:04.057 |[   base.py:252] Thread-33 (listen) | Listening for MAVLink commands from system: 200...
INFO   | uav.CamClient        | 20:20:04.058 |[   base.py:105] MainThread | Starting MAVLink connection... Mavlink version 2 = True
INFO   | uav.CamClient        | 20:20:04.060 |[   base.py:117] MainThread | Source system Set: 200, Source component: 25
INFO   | uav.CamClient        | 20:20:04.061 |[   base.py:118] MainThread | Target system Set: 1, Target component: 100
INFO   |

20


INFO   | uav.CamClient        | 20:20:05.063 |[   base.py:285] Thread-34 (listen) | Stopped
INFO   | uav.CamClient        | 20:20:05.098 |[   base.py:347] MainThread | CamClient  closed
INFO   | uav.CamServer        | 20:20:05.152 |[   base.py:285] Thread-33 (listen) | Stopped
INFO   | uav.CamServer        | 20:20:06.062 |[   base.py:347] MainThread | CamServer  closed


server.num_commands_received: 3
client.num_commands_sent: 3
client.num_acks_received: 3
server msgs: {'BAD_DATA': 2, 'HEARTBEAT': 2, 'COMMAND_LONG': 3}
client msgs: {'BAD_DATA': 20, 'TIMESYNC': 9, 'HEARTBEAT': 78, 'PARAM_VALUE': 3, 'STATUSTEXT': 16, 'COMMAND_ACK': 3}


In [ ]:
# assert False, "Stop here"

> Connection using a serial crossover cable or via pixhawk telemetry ports
![](../tutorials/images/serial_crossover.jpeg)  
Telemetry 2 port on pixhawk is connected to the USB port on the companion computer using a serial crossover cable.
1. (red) VCC +5V
2. (?) TX (OUT) +3.3V
3. (?) RX (IN) +3.3V
4. (?) CTS +3.3V
5. (?) RTS +3.3V
6. (?) GND GND`

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()